# Convert audio files to melspectrograms to be used as input images to the image classification algorithm

### This process will take the 3 second audio segments and create a spectral visual representation of them

## Step 1: Install dependencies 

In [ ]:
!conda install -c conda-forge librosa -y

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.

#### If you have trouble installing librosa using the above command, try the following alternative:

!conda install -c conda-forge librosa -y
!conda install -c conda-forge libsndfile -y
!pip install soundfile

In [ ]:
import os 
import matplotlib.pyplot as plt
import librosa
import librosa.display
import warnings
import numpy as np

#### Suppress FutureWarnings because the current version of librosa throws many of these

In [61]:
warnings.simplefilter(action='ignore', category=FutureWarning)

### Step 2: These are our functions to create the spectogram and melspectograms 

For our methodology we have chosen to use the melspectrogram method

We have configured matplotlib to output each gram as 224 x 224 pixels

In [76]:
def make_spgram(file_path, class_name): 
    base = os.path.basename(file_path)
    #need subfolder too 
    input_file = os.path.splitext(base)[0]

    clip, sample_rate = librosa.load(file_path, sr=None)
    n_fft = 1024 # frame length 
    
    hop_length = 512
    stft = librosa.stft(clip, n_fft=n_fft, hop_length=hop_length)
    stft_magnitude, stft_phase = librosa.magphase(stft)
    stft_magnitude_db = librosa.amplitude_to_db(stft_magnitude, ref=np.max)

    plt.figure(figsize=(4, 4))

    librosa.display.specshow(stft_magnitude_db, x_axis=None, y_axis=None, 
                             sr=sample_rate, hop_length=hop_length)

    plt.savefig('../data/spectrogram/'+class_name+'/'+input_file+'.png',bbox_inches='tight',pad_inches=0)
    plt.clf()

In [74]:
def make_melspgram(file_path, species, data_split):
    n_mels = 64
    base = os.path.basename(file_path)
    input_file = os.path.splitext(base)[0]
    clip, sample_rate = librosa.load(file_path, sr=None)
    n_fft = 1024 # frame length 
    fmin = 0            
    fmax = 22050 # sample_rate/2
    hop_length = 512
    mel_spec = librosa.feature.melspectrogram(clip, n_fft=n_fft, hop_length=hop_length,
                                              n_mels=n_mels, sr=sample_rate, power=1.0,
                                              fmin=fmin, fmax=fmax)
    mel_spec_db = librosa.amplitude_to_db(mel_spec, ref=np.max)
    plt.figure(figsize=(4, 4))
    librosa.display.specshow(mel_spec_db, x_axis=None,  y_axis=None, 
                             sr=sample_rate, hop_length=hop_length, 
                             fmin=fmin, fmax=fmax)    
    
    plt.savefig('../data/mel-spectrogram/'+data_split+'/'+species+ '/'+input_file+'.png',bbox_inches='tight',pad_inches=0)
    plt.show()
    plt.close()

### Step 3: apply it to the images, by going through the folders and sub-folders 

In [ ]:
#if don't get im2rec working with the splitting do this: 
bird_species = ['eastern-whipbird', 'kookaburra', 'willie-wagtail']
rootdir = '../data/clips'

count = 0
for i in bird_species: 
    input_dir = rootdir+i
    if not os.path.isdir('../data/mel-spectrogram/validate'):
        os.mkdir('../data/mel-spectrogram/validate')
    if not os.path.isdir('../data/mel-spectrogram/train'):
        os.mkdir('../data/mel-spectrogram/train')

    for subdir, dirs, files in os.walk(input_dir):
        for file in files:
            file_path = os.path.join(subdir, file)
            count += 1
            if count%5 == 0:
                if not os.path.isdir('../data/mel-spectrogram/validate'):
                    os.mkdir('../data/mel-spectrogram/validate')
                make_melspgram(file_path, i, 'validate')
            elif count%5 != 0:
                if not os.path.isdir('../data/mel-spectrogram/train'):
                    os.mkdir('../data/mel-spectrogram/train')
                make_melspgram(file_path, i, 'train')

## Write to S3 

In [ ]:
import boto3
import os
s3r = boto3.resource('s3')
bucket = s3r.Bucket('audio-classifier-data')

path = '../Data/mel-spectrogram/train'
for subdir, dirs, files in os.walk(path):
    for file in files:
        full_path = os.path.join(subdir, file)
        #with the moving of the data, needed to chagne the path for s3
        image_path = '/'.join(full_path.split('/')[2:]))
        with open(full_path, 'rb') as data:
            bucket.put_object(Key=image_path, Body=data)

In [82]:
path = '../Data/mel-spectrogram/validate'
for subdir, dirs, files in os.walk(path):
    for file in files:
        full_path = os.path.join(subdir, file)
        #with the moving of the data, needed to chagne the path for s3
        image_path = '/'.join(full_path.split('/')[2:]))
        with open(full_path, 'rb') as data:
            bucket.put_object(Key=image_path, Body=data)

mel-spectrogram/validate/bird-audio-validate.lst
mel-spectrogram/validate/eastern-whipbird/b1a5da01-8a74-4abb-8cce-c68c31dc0a60_stereo_10000_13000.png
mel-spectrogram/validate/eastern-whipbird/aad78fc7-f674-4d44-af53-d3d25083420e_3500_6500-checkpoint.png
mel-spectrogram/validate/eastern-whipbird/b1a5da01-8a74-4abb-8cce-c68c31dc0a60_stereo_14000_17000.png
mel-spectrogram/validate/eastern-whipbird/bf8d74b6-9763-4978-af5e-dda940d8f5f1_3000_6000-checkpoint.png
mel-spectrogram/validate/eastern-whipbird/b1a5da01-8a74-4abb-8cce-c68c31dc0a60_stereo_6500_9500.png
mel-spectrogram/validate/eastern-whipbird/b1a5da01-8a74-4abb-8cce-c68c31dc0a60_stereo_3000_6000.png
mel-spectrogram/validate/eastern-whipbird/aad78fc7-f674-4d44-af53-d3d25083420e_1500_4500-checkpoint.png
mel-spectrogram/validate/eastern-whipbird/e409f4db-c60c-4d94-afbe-e7c3ba8db79c_stereo_1500_4500.png
mel-spectrogram/validate/eastern-whipbird/e409f4db-c60c-4d94-afbe-e7c3ba8db79c_stereo_1000_4000.png
mel-spectrogram/validate/eastern-wh